In [1]:
import torch
import numpy as np
from scipy.io import wavfile
import librosa
from torchaudio.transforms import Spectrogram, InverseSpectrogram

In [2]:
dir = "D:/Projects/LCT-GAN/.data"

model = torch.jit.load(f'{dir}/FTFNet_scripted.pt')
model.eval()
device = torch.device("cpu")

In [ ]:
model.state_dict()

In [3]:
model

RecursiveScriptModule(
  original_name=TorchWrapper
  (model): RecursiveScriptModule(
    original_name=FTFNet
    (conv1): RecursiveScriptModule(original_name=Conv2d)
    (conv2): RecursiveScriptModule(original_name=Conv2d)
    (conv3): RecursiveScriptModule(original_name=Conv2d)
    (deconv2): RecursiveScriptModule(original_name=ConvTranspose2d)
    (deconv3): RecursiveScriptModule(original_name=ConvTranspose2d)
    (deconv4): RecursiveScriptModule(original_name=ConvTranspose2d)
    (activation): RecursiveScriptModule(original_name=LeakyReLU)
    (activations): RecursiveScriptModule(original_name=Sigmoid)
    (skip2): RecursiveScriptModule(original_name=Conv2d)
    (skip3): RecursiveScriptModule(original_name=Conv2d)
    (skip4): RecursiveScriptModule(original_name=Conv2d)
    (GRUf1): RecursiveScriptModule(
      original_name=GRUblockf
      (gru1): RecursiveScriptModule(original_name=GRU)
      (gru2): RecursiveScriptModule(original_name=GRU)
      (gru3): RecursiveScriptModule(or

In [4]:
print(model.code)

def forward(self,
    inputs: Tensor) -> Tensor:
  inputs0 = torch.unsqueeze(inputs, 0)
  inputs_mag = torch.pow(torch.abs(inputs0), 0.29999999999999999)
  model = self.model
  mask_preds = (model).forward(inputs_mag, )
  outputs = torch.mul(inputs_mag, mask_preds)
  outputs0 = torch.pow(outputs, 3.3333333333333335)
  _0 = torch.mul(torch.angle(inputs0), 0.+1.j)
  outputs1 = torch.mul(outputs0, torch.exp(_0))
  _1 = torch.squeeze(torch.permute(outputs1, [0, 1, 3, 2]))
  return _1



In [5]:
model.graph

graph(%self.1 : __torch__.___torch_mangle_11.TorchWrapper,
      %inputs.1 : Tensor):
  %47 : complex = prim::Constant[value=0.+1.j]()
  %3 : int = prim::Constant[value=0]() # /tmp/ipykernel_1855/2753369034.py:21:34
  %7 : float = prim::Constant[value=0.29999999999999999]() # /tmp/ipykernel_1855/2753369034.py:22:50
  %18 : float = prim::Constant[value=3.3333333333333335]() # :0:0
  %31 : int = prim::Constant[value=1]() # /tmp/ipykernel_1855/2753369034.py:27:37
  %32 : int = prim::Constant[value=3]() # /tmp/ipykernel_1855/2753369034.py:27:40
  %33 : int = prim::Constant[value=2]() # /tmp/ipykernel_1855/2753369034.py:27:43
  %inputs0.1 : Tensor = aten::unsqueeze(%inputs.1, %3) # /tmp/ipykernel_1855/2753369034.py:21:17
  %6 : Tensor = aten::abs(%inputs0.1) # /tmp/ipykernel_1855/2753369034.py:22:31
  %inputs_mag.1 : Tensor = aten::pow(%6, %7) # /tmp/ipykernel_1855/2753369034.py:22:21
  %model.1 : __torch__.lct_la1n.FTFNet = prim::GetAttr[name="model"](%self.1)
  %mask_preds.1 : Tensor = pr

In [7]:
for name, m in model.named_modules():
    print(name, ":", m.__class__.__name__)

 : RecursiveScriptModule
model : RecursiveScriptModule
model.conv1 : RecursiveScriptModule
model.conv2 : RecursiveScriptModule
model.conv3 : RecursiveScriptModule
model.deconv2 : RecursiveScriptModule
model.deconv3 : RecursiveScriptModule
model.deconv4 : RecursiveScriptModule
model.activation : RecursiveScriptModule
model.activations : RecursiveScriptModule
model.skip2 : RecursiveScriptModule
model.skip3 : RecursiveScriptModule
model.skip4 : RecursiveScriptModule
model.GRUf1 : RecursiveScriptModule
model.GRUf1.gru1 : RecursiveScriptModule
model.GRUf1.gru2 : RecursiveScriptModule
model.GRUf1.gru3 : RecursiveScriptModule
model.GRUf1.gru4 : RecursiveScriptModule
model.GRUf1.attn : RecursiveScriptModule
model.GRUf1.attn.out_proj : RecursiveScriptModule
model.GRUf1.activationtrans : RecursiveScriptModule
model.GRUf1.layernorm1 : RecursiveScriptModule
model.GRUf1.layernorm2 : RecursiveScriptModule
model.GRUf1.lin : RecursiveScriptModule
model.GRUf2 : RecursiveScriptModule
model.GRUf2.gru1 : 

In [6]:
for name, p in model.state_dict().items():
    print(name, p.shape)

model.conv1.weight torch.Size([16, 1, 2, 3])
model.conv1.bias torch.Size([16])
model.conv2.weight torch.Size([32, 16, 2, 3])
model.conv2.bias torch.Size([32])
model.conv3.weight torch.Size([64, 32, 2, 3])
model.conv3.bias torch.Size([64])
model.deconv2.weight torch.Size([64, 32, 2, 3])
model.deconv2.bias torch.Size([32])
model.deconv3.weight torch.Size([32, 16, 2, 3])
model.deconv3.bias torch.Size([16])
model.deconv4.weight torch.Size([16, 1, 2, 3])
model.deconv4.bias torch.Size([1])
model.skip2.weight torch.Size([64, 1, 1, 1])
model.skip2.bias torch.Size([64])
model.skip3.weight torch.Size([32, 1, 1, 1])
model.skip3.bias torch.Size([32])
model.skip4.weight torch.Size([16, 1, 1, 1])
model.skip4.bias torch.Size([16])
model.GRUf1.gru1.weight_ih_l0 torch.Size([48, 16])
model.GRUf1.gru1.weight_hh_l0 torch.Size([48, 16])
model.GRUf1.gru1.bias_ih_l0 torch.Size([48])
model.GRUf1.gru1.bias_hh_l0 torch.Size([48])
model.GRUf1.gru1.weight_ih_l0_reverse torch.Size([48, 16])
model.GRUf1.gru1.weight

In [ ]:
audio_file = 'D:/Projects/LCT-GAN/.data/noisy_4.wav'
framelen = 512
hoplen = 256
win = lambda x: torch.sqrt(torch.hann_window(x)).to(device) # sqrt-hann window
to_spec = Spectrogram(n_fft=framelen, hop_length=hoplen, power=None, window_fn=win)
from_spec = InverseSpectrogram(n_fft=framelen, hop_length=hoplen, window_fn=win)

x, _ = librosa.load(audio_file, sr=16000)
x = torch.tensor(x.astype(np.float32)).to(device).unsqueeze(0)
inputs = to_spec(x).permute(0,2,1).cfloat() # from waveform to spectrogram (channel, T, F)

In [ ]:
from torchviz import make_dot

model.eval()

output = model(inputs)

if isinstance(output, (list, tuple)):
    output = output[0]

dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = "png"
dot.render("ftfnet_graph")

print("Saved computation graph as ftfnet_graph.png")